In [279]:
import collections
from itertools import permutations, combinations, combinations_with_replacement, product
from scipy.special import comb, binom
from scipy.stats import norm
from math import factorial
import random
import numpy as np
import pandas as pd
import time

### Poker:

In this section, we explore the game of Poker. Specifically we are interested in the chance of obtaining a specific poker hand when we draw 5 cards (a hand) from a deck of well-shuffled playing cards.

#### The ranking of Poker hands (in decreasing order):
1. Royal Flush: A, K, Q, J, 10 all same suit
2. Straight Flush: 5 consecutive cards all same suit
3. Four of a Kind: 4 cards of same rank
4. Full House: 3 cards of one rank + 2 cards of another (same) rank
5. Flush: Any five cards of same suit but not in a sequence
6. Straight: Five cards in a sequence, not of same suit
7. Three of a kind: 3 cards of same rank
8. Two pair: 2 cards of one rank (same) + 2 cards of another (same) rank
9. One pair: 2 cards of same rank

Ref: [https://www.cardplayer.com/rules-of-poker/hand-rankings]

### Simulating a deck of playing cards 

1. A standard deck of playiong cards consists of 52 cards in total
2. There are 4 suits: Hearts (H), Diamonds (D), Clubs (C), Spades (S)
3. Each suit has a set of 13 ranks:
    a. Numbered cards: 2 to 10
    b. Face cards: J, Q, K
    c. Ace
4. The Ace doubles up as vales=1 and also the highest order rank with value = 14

The sequence of Cards in increasing order of ranking (Ace repeats twice):
**A, 2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K, A**

Below is code to simulate/define a deck of cards *(see Fluent Python L. Ramalho)*

In [354]:
# Pythonic deck 

ranks = [str(x) for x in range(2,11)] + list('JQKA')
suits = list('HDCS')
values = dict(zip(range(1, 15),['A']+ranks))

deck = list(product(ranks, suits))    # as a list of rank and suit

# A more organized way to build the deck using Named tuples

Card = collections.namedtuple('Card', ['rank', 'suit'])
py_deck = [Card(rank, suit) for rank in ranks for suit in suits]

In [355]:
comb(52, 5)

2598960.0

## Computing probabilities Analytically

Total number of possible combinations
$\bigl( \begin{smallmatrix} 52 \\ 5 \end{smallmatrix} \bigr) = 2,598,960$

**Probability of Royal Flush**

*Royal Flush: A, K, Q, J, 10 all same suit*

Number of suits available to choose from: $\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr) = 4$

Each suits has one and only one royal straight range 

Total number of favorable of outcomes = $\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr)$

$
p(royal flush) = $${\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr)}\over\bigl(\begin{smallmatrix}52\\5\end{smallmatrix}\bigr)$$
$


**Probability of Straight Flush**

*Straight Flush: 5 consecutive cards, all same suit*

Number of suits available to choose from: $\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr) = 4$

For each suit we need to compute the # of possible straight sequences
Keeping in mind that A can double up as both value=1 and value=14

First straight sequence = A, 2, 3, 4, 5
Second straight sequence = 2, 3, 4, 5, 6
.
.
Last straight sequence = 10, J, Q, K, A

Out of the 14 card values in the range, and there is one and only one straight sequence possible starting with each of the cards in the sequence A, 2, 3, ..., 10 = Total of 10 straight sequences

Hence, total number of favorable of outcomes = $\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr)*10$

$
p(straight flush) = $${\bigl(\begin{smallmatrix}4\\1\end{smallmatrix}\bigr)}*10\over\bigl(\begin{smallmatrix}52\\5\end{smallmatrix}\bigr)$$
$

**Probability of Four of a kind**
*Four of a Kind: 4 cards of same rank*

Number of ranks to pick from: $\bigl(\begin{smallmatrix}13\1\end{smallmatrix}\bigr) = 13$
Within the chosen rank there are 4 possible suits to pick from: $\bigl(\begin{smallmatrix}4\4\end{smallmatrix}\bigr) = 1$

There is still on card to choose form the remaining: 52-4 = 48

Total number of favorable of outcomes = $\bigl(\begin{smallmatrix}13\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\4\end{smallmatrix}\bigr)*48$

$
p(four of a kind) = $$\bigl(\begin{smallmatrix}13\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\4\end{smallmatrix}\bigr)*48\over\bigl(\begin{smallmatrix}52\\5\end{smallmatrix}\bigr)$$
$

**Probability of Full House**
*Full House: 3 cards of same rank + 2 cards of same rank*

Number of ways to select the rank with 3 cards: $\bigl(\begin{smallmatrix}13\1\end{smallmatrix}\bigr) = 13$
From this chosen rank, number of ways to choose three cards from the available 4 suits:$\bigl(\begin{smallmatrix}4\3\end{smallmatrix}\bigr) = 4$

Number of ways to select the rank with 2 cards from remaining: $\bigl(\begin{smallmatrix}12\1\end{smallmatrix}\bigr) = 12$
From this chosen rank, number of ways to choose two cards from the available 4 suits:$\bigl(\begin{smallmatrix}4\2\end{smallmatrix}\bigr) = 6$

Total nunmber of favorable of outcomes = $\bigl(\begin{smallmatrix}13\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\3\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}12\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\2\end{smallmatrix}\bigr)$

$
p(full house) = $$\bigl(\begin{smallmatrix}13\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\3\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}12\\1\end{smallmatrix}\bigr)*\bigl(\begin{smallmatrix}4\\2\end{smallmatrix}\bigr)\over\bigl(\begin{smallmatrix}52\\5\end{smallmatrix}\bigr)$$
$

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of flush
 Flush: Any five cards of same suit but not in a sequence
 Number of ways to choose a suit from available = comb(4,1)
 Number of ways to choose 5 cards from ranks available from the chosen suit = comb(13, 5)
 Number of combinations having a straigh sequence from this list of sequences = 10
 Number of hands withoout a straigh sequence = (comb(13, 5)-10)

 total nunmber of favorable of outcomes = comb(13, 5)*(comb(13, 5)-10)

p_flush = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of straight
 Straight: Five cards in a sequence, not of same suit

 A total number of 10 straight sequences are possible each starting with A, 2, ...., 10
 For each of these sequences, there are a total of 4**5 combinations of suits possible
 However, one of the combinations would be a straigh flush, i.e. all cards from the same suit
 There are 4 such combinations possible, 1 for each suit
 Hence total number of suit combinations for each sequence excluding the flush is 4**5-4

 total nunmber of favorable of outcomes = 10*(4**5-4)

p_straight = 10*(4**5-4)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of three_kind
 Three of a kind: 3 cards of same rank
 Number of ways to choose a rank for the 3 cards = comb(13,1)
 Number of ways to choose 3 cards from 4 for the selected rank = comb(4,3)
 Number of ways to choose 2 cards from the remaining ranks = comb(12*4,2)
 total nunmber of favorable of outcomes = comb(13,1)*comb(4,3)*comb(12*4,2)

p_three_kind = comb(13,1)*comb(4,3)*comb(12*4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of two_pair
 Two pair: 2 cards of same rank + 2 cards of another rank
 Number of ways to choose 2 rank for the 4 cards = comb(13,2)
 Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
 Number of ways to choose 1 cards from the remaining ranks = comb(11*4,1)

total nunmber of favorable of outcomes = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)

p_two_pair = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of one_pair
 One pair: 2 cards of same rank

 Number of ways to choose 1 rank for the 2 cards = comb(13,1)
 Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
 Number of ways to choose 3 cards from the remaining ranks such that they do not contain another pair = (12*4)*(11*4)*(10*4)
 The other three cards cam be permuted in 3! ways amongst themselves

 total nunmber of favorable of outcomes = comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3))

p_one_pair = (comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3)))/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
 Probability of not_special
 Note special: none of the above

p_not_special = 1-sum([p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair])

#-----------------------------------------------------------------------------------------------------------------------------
 Hands index probabilities

hands_index_ana_probs = [p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair, p_not_special]

In [282]:
# generating a random draw of five cards - "A Poker Hand"
hand = random.sample(py_deck,k=5) # unordered selection of 5 cards without repacement

In [283]:
# Counting the number of ranks and suits in a given hand
def get_counters(hand):
    c_ranks = collections.Counter([a[0] for a in hand])
    c_suits = collections.Counter([a[1] for a in hand])
    c_values = collections.Counter([key for key,val in values.items() if val in c_ranks])
    c_values_noA = collections.Counter([key for key,val in values.items() if (val in c_ranks) & (val!='A')])
    return c_ranks, c_suits, c_values, c_values_noA

#### Order of Poker hands:
1. Royal Flush: A, K, Q, J, 10 all same suit
2. Straight Flush: 5 consecutive cards all same suit
3. Four of a Kind: 4 cards of same rank
4. Full House: 3 cards of same rank + 2 another of same rank
5. Flush: Any five cards of same suit but not in a sequence
6. Straight: Five cards in a sequence, not of same suit
7. Three of a kind: 3 cards of same rank
8. Two pair: 2 cards of same rank + 2 cards of same rank
9. One pair: 2 cards of same rank

In [349]:
## Key learning: Defining the condition for straight
# Incorrect: (((_A_in_ranks)&(_val_range_noA==3))|((~_A_in_ranks)&(_val_range_wA==4)))
# Just the range being 4 will not work as there could be 2,2,2,2,6 will also satify the range condition

# Even this is not enough:
#     hand = [Card(rank='6', suit='H'),
#  Card(rank='7', suit='H'),
#  Card(rank='8', suit='H'),
#  Card(rank='9', suit='H'),
#  Card(rank='A', suit='C')]

# There are only two possible straight sequences with A in them, just hardcode them for now

In [350]:
def evaluator(h):
    _c_ranks, _c_suits, _c_values, _c_values_noA = get_counters(h)
    ## Individual conditions
    _royal_condn = (_c_ranks == collections.Counter(['A', 'K', 'Q', 'J', '10']))
    _flush_condn = (_c_suits.most_common(1)[0][1] == 5)
    _A_in_ranks = ('A' in _c_ranks)
    _straight_wA = ((1,2,3,4,5,14), (1,10,11,12,13,14))
    _val_range_noA = (max(_c_values_noA.keys())-min(_c_values_noA.keys()))
    _count_vals_noA = len(_c_values_noA.keys())
    _straight_condn = (
                        ((_A_in_ranks)&(tuple(_c_values.keys()) in _straight_wA))|
                        ((~_A_in_ranks)&(_val_range_noA==4)&(_count_vals_noA==5))
                      )
    _most_common_rank1, _most_common_rank2 = _c_ranks.most_common(1)[0][1], _c_ranks.most_common(2)[1][1]
    _rank1_gt4, _rank1_gt3, _rank1_gt2 = (_most_common_rank1 >= 4), (_most_common_rank1 >= 3), (_most_common_rank1 >= 2)
    _rank2_gt2 = (_most_common_rank2 >= 2)
    ## Special Hand Conditions
    _special_hands = collections.OrderedDict([('royal_flush', (_royal_condn&_flush_condn))
                                         , ('straight_flush', (_flush_condn&_straight_condn))
                                         , ('four_kind', _rank1_gt4)
                                         , ('full_house', (_rank1_gt3&_rank2_gt2))
                                         , ('flush', (_flush_condn&~_straight_condn))
                                         , ('straight', (_straight_condn&~_flush_condn))
                                         , ('three_kind', _rank1_gt3)
                                         , ('two_pair', (_rank1_gt2&_rank2_gt2))
                                         , ('one_pair', _rank1_gt2)
                                         ,   
                                        ])
    for _ in _special_hands:
        if _special_hands[_]:
            return _
    else:
        return 'not_special'

In [351]:
def get_hands(num_hands, deck=py_deck):
    hands = []
    for i in range(num_hands):
        hand = random.sample(py_deck,k=5)
        hands.append(hand)
    return hands

In [352]:
def special_hand_counter(hands, prob=False):
    _accum = []
    for h in hands:
        val = evaluator(h)
        _accum.append(val)
    counts = collections.Counter(_accum)
    total = sum(counts.values())
    if prob:
        d={}
        for k, v in counts.items():
            d[k] = v/total
        return d
    else:
        return counts

In [353]:
# Compute probabilities of eah of the above hands through simulation
# Verify with analytically computed probabilities where possible

## Running the n simulation
    # Each round of simulation draws 1Mn hands from the deck
hands_index = ['royal_flush', 'straight_flush', 'four_kind', 'full_house'
               , 'flush', 'straight', 'three_kind', 'two_pair', 'one_pair', 'not_special']
sim_df = pd.DataFrame(hands_index, columns=['hand_ranks'])
start_time = time.time()
n = 10    #number of simulations
n_hands = int(5e6)    #number of hands in each simulation 
for i in range(n):
    sim_num = 'Sim_'+str(i)
    print(f'Starting {sim_num} run now.')
    sim_start_time = time.time()
    H = get_hands(num_hands=n_hands)
    sp_count = special_hand_counter(H, prob=True)
    sp_count = pd.DataFrame({'hand_ranks':sp_count.keys(), sim_num:sp_count.values()})
    sim_df = sim_df.merge(sp_count, on='hand_ranks', how='left')
    sim_end_time = time.time()
    print(f'{sim_num} ended. This run took {sim_end_time-sim_start_time} seconds to complete')
df = sim_df.copy()
end_time = time.time()
print(f'{n} simulations took a total of {end_time-start_time} seconds')

Starting Sim_0 run now.
Sim_0 ended. This run took 217.05060601234436 seconds to complete
Starting Sim_1 run now.
Sim_1 ended. This run took 207.71397995948792 seconds to complete
Starting Sim_2 run now.
Sim_2 ended. This run took 209.43329858779907 seconds to complete
Starting Sim_3 run now.
Sim_3 ended. This run took 208.89683842658997 seconds to complete
Starting Sim_4 run now.
Sim_4 ended. This run took 210.53937673568726 seconds to complete
Starting Sim_5 run now.
Sim_5 ended. This run took 204.6105444431305 seconds to complete
Starting Sim_6 run now.
Sim_6 ended. This run took 205.8545160293579 seconds to complete
Starting Sim_7 run now.
Sim_7 ended. This run took 203.9000813961029 seconds to complete
Starting Sim_8 run now.
Sim_8 ended. This run took 207.69504356384277 seconds to complete
Starting Sim_9 run now.
Sim_9 ended. This run took 189.88436031341553 seconds to complete
10 simulations took a total of 2065.5806407928467 seconds


In [345]:
df['avg_probs'] = df[[col for col in df.columns if 'Sim' in col]].mean(axis=1)
df['std_err_probs'] = np.sqrt(df['avg_probs']*(1-df['avg_probs'])/(n*num_hands))

# df['avg_probs_lower'], df['avg_probs_upper'] = norm.interval(0.95, loc=df['avg_probs'], scale=df['std_err_probs'])  # Careful!!, may give negative probabilities
# df['avg_probs_lower'], df['avg_probs_upper'] = np.maximum(df['avg_probs_lower'], 0), np.maximum(df['avg_probs_upper'], 0)

z_CI_lower, z_CI_upper = norm.interval(0.95, loc=0, scale=1)
df['avg_probs_lower'] = np.maximum(df['avg_probs']+z_CI_lower*df['std_err_probs'], 0)
df['avg_probs_upper'] = np.maximum(df['avg_probs']+z_CI_upper*df['std_err_probs'], 0)

In [346]:
## Analytically compute probabilities of each special hand

## Total number of possible combinations
total_comb = comb(52, 5)

## Probability of royal_flush
# Royal Flush: A, K, Q, J, 10 all same suit

# of suits to choose for the royal straigh range = comb(4,1)

# total nunmber of favorable of outcomes = comb(4,1)

p_royal_flush = comb(4,1)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
## Probability of straight_flush
# Straight Flush: 5 consecutive cards all same suit

# of suits to choose for the straight range = 4C1 = comb(4, 1)
# for each suit we need to compute the # of possible straight sequences
# A can double up as both value=1 and value=14
## First straight sequence = A, 2, 3, 4, 5
## Last straight sequence = 10, J, Q, K, A
## Total 14 cards in full range, and there is one and only one straight range possible starting with each of the cards in
## the sequence A,2,3,...,10 - Total 10 sequences

# total nunmber of favorable of outcomes = comb(4,1)*10

# total possibilities = total comb
p_straight_flush = comb(4,1)*10/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of four_kind
# Four of a Kind: 4 cards of same rank

# of ranks to pick from = comb(13,1)
# of cards to pick from a particular suit = comb(4,4)
# of possibilities for the fifth card = 52-4 = 48

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,4)*48

p_four_kind = comb(13,1)*comb(4,4)*48/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of full_house
# Full House: 3 cards of same rank + 2 cards of same rank

## Need to have 3 cards of a kind and 2 cards of a different kind

## of ways to select the rank having 3 cards = comb(13,1)
## Of this particular rank, number of ways to choose three cards from the available 4 suits = comb(4,3)
## of ways to select the rank having 2 cards from the remaining ranks = comb(12,1)
## Of this particular rank, number of ways to choose two cards from the available 4 suits = comb(4,2)

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)

p_full_house = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of flush
# Flush: Any five cards of same suit but not in a sequence

# Number of ways to choose a suit from available = comb(4,1)
# Number of ways to choose 5 cards from ranks available from the chosen suit = comb(13, 5)
# Number of combinations having a straigh sequence from this list of sequences = 10
# Number of hands withoout a straigh sequence = (comb(13, 5)-10)

# total nunmber of favorable of outcomes = comb(13, 5)*(comb(13, 5)-10)

p_flush = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of straight
# Straight: Five cards in a sequence, not of same suit

# A total number of 10 straight sequences are possible each starting with A, 2, ...., 10
# For each of these sequences, there are a total of 4**5 combinations of suits possible
# However, one of the combinations would be a straigh flush, i.e. all cards from the same suit
# There are 4 such combinations possible, 1 for each suit
# Hence total number of suit combinations for each sequence excluding the flush is 4**5-4

# total nunmber of favorable of outcomes = 10*(4**5-4)

p_straight = 10*(4**5-4)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of three_kind
# Three of a kind: 3 cards of same rank

# Number of ways to choose a rank for the 3 cards = comb(13,1)
# Number of ways to choose 3 cards from 4 for the selected rank = comb(4,3)
# Number of ways to choose 2 cards from the remaining ranks = comb(12*4,2)

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,3)*comb(12*4,2)

p_three_kind = comb(13,1)*comb(4,3)*comb(12*4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of two_pair
# Two pair: 2 cards of same rank + 2 cards of another rank

# Number of ways to choose 2 rank for the 4 cards = comb(13,2)
# Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
# Number of ways to choose 1 cards from the remaining ranks = comb(11*4,1)

# total nunmber of favorable of outcomes = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)

p_two_pair = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of one_pair
# One pair: 2 cards of same rank

# Number of ways to choose 1 rank for the 2 cards = comb(13,1)
# Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
# Number of ways to choose 3 cards from the remaining ranks such that they do not contain another pair = (12*4)*(11*4)*(10*4)
# The other three cards cam be permuted in 3! ways amongst themselves

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3))

p_one_pair = (comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3)))/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of not_special
# Note special: none of the above

p_not_special = 1-sum([p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair])

#-----------------------------------------------------------------------------------------------------------------------------
# Hands index probabilities

hands_index_ana_probs = [p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair, p_not_special]

In [347]:
df_ana = pd.DataFrame(zip(hands_index, hands_index_ana_probs), columns=['hand_ranks', 'mu_analytical_probs'])
df_ana    # probability of getting a royal flush is 2 in a million, hence number of draws per simulation has to be atleast 5Mn 

,hand_ranks,mu_analytical_probs
0,royal_flush,0.000002
1,straight_flush,0.000015
2,four_kind,0.000240
3,full_house,0.001441
4,flush,0.001441
5,straight,0.003925
6,three_kind,0.022569
7,two_pair,0.047539
8,one_pair,0.422569
9,not_special,0.500260


In [348]:
df = df.merge(df_ana, on='hand_ranks', how='left')
df['mu_in_CI'] = ((df.mu_analytical_probs>=df.avg_probs_lower)|(df.mu_analytical_probs<=df.avg_probs_upper))
df

,hand_ranks,Sim_0,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,Sim_7,Sim_8,Sim_9,avg_probs,std_err_probs,avg_probs_lower,avg_probs_upper,mu_analytical_probs,mu_in_CI
0,royal_flush,0.000004,NaN,0.000002,0.000002,0.000002,NaN,NaN,0.000002,0.000004,NaN,0.000003,5.163971e-07,0.000002,0.000004,0.000002,True
1,straight_flush,0.000026,0.000018,0.000012,0.000014,0.000012,0.000012,0.000020,0.000020,0.000006,0.000012,0.000015,1.232873e-06,0.000013,0.000018,0.000015,True
2,four_kind,0.000176,0.000258,0.000288,0.000270,0.000246,0.000302,0.000216,0.000234,0.000230,0.000234,0.000245,4.953179e-06,0.000236,0.000255,0.000240,True
3,full_house,0.001402,0.001372,0.001472,0.001436,0.001278,0.001422,0.001372,0.001340,0.001418,0.001396,0.001391,1.178501e-05,0.001368,0.001414,0.001441,True
4,flush,0.001838,0.001922,0.001962,0.001880,0.001950,0.001978,0.001904,0.001888,0.002012,0.001926,0.001926,1.386467e-05,0.001899,0.001953,0.001441,True
5,straight,0.003842,0.003902,0.003778,0.003976,0.003884,0.003802,0.003878,0.003974,0.003980,0.003952,0.003897,1.970181e-05,0.003858,0.003935,0.003925,True
6,three_kind,0.021000,0.021598,0.020982,0.020996,0.021332,0.021556,0.021190,0.021094,0.021042,0.021380,0.021217,4.557065e-05,0.021128,0.021306,0.022569,True
7,two_pair,0.047968,0.047814,0.047672,0.047740,0.047240,0.047132,0.047352,0.046962,0.047382,0.048126,0.047539,6.728957e-05,0.047407,0.047671,0.047539,True
8,one_pair,0.423198,0.422332,0.422278,0.422856,0.423824,0.421632,0.423738,0.423158,0.423808,0.421542,0.422837,1.562197e-04,0.422530,0.423143,0.422569,True
9,not_special,0.500546,0.500784,0.501554,0.500830,0.500232,0.502164,0.500330,0.501328,0.500118,0.501432,0.500932,1.581136e-04,0.500622,0.501242,0.500260,True


In [ ]:
# Create an Evaluator class to do the checking
# Draw back if that it may take up too much space and may be slower than what we need